In [2]:
import xlwings as xw

# Create a new Excel workbook
wb = xw.Book()

# Use the first sheet and rename it
sheet = wb.sheets[0]
sheet.name = "LiveData"

# Set column headers
sheet.range("A1:D1").value = ["Symbol", "LTP", "Bid Qty", "Ask Qty"]

# Save the workbook
wb.save("live_data.xlsx")
wb.close()

print("✅ Excel file 'live_data.xlsx' created successfully.")


✅ Excel file 'live_data.xlsx' created successfully.


In [14]:
import os
import pandas as pd
from datetime import timedelta

# === CONFIG ===
FOLDER = "NIFTY50 JUN19"
LOOKBACK_WINDOW = 5  # in minutes
HOLDING_PERIOD = 1   # exit after 1 minute
results = []

def load_tick_data(file_path):
    df = pd.read_csv(file_path, parse_dates=["timestamp"])
    df = df[["timestamp", "ltp", "bid_qty", "ask_qty"]]
    df = df.sort_values("timestamp")
    df = df.set_index("timestamp")
    return df

def simulate_ltp_breakouts(df, symbol):
    trades = []
    timestamps = df.index.unique().sort_values()

    for i in range(LOOKBACK_WINDOW * 60, len(timestamps) - HOLDING_PERIOD * 60):
        current_time = timestamps[i]
        past_window = df.loc[timestamps[i - LOOKBACK_WINDOW * 60]:timestamps[i]]

        if len(past_window) < 10:
            continue

        high = past_window['ltp'].max()
        low = past_window['ltp'].min()
        now = df.loc[current_time]

        ltp = now['ltp']
        bid_qty = now['bid_qty']
        ask_qty = now['ask_qty']

        # LTP breakout
        if ltp > high and bid_qty > ask_qty:
            exit_time = timestamps[i + HOLDING_PERIOD * 60]
            exit_price = df.loc[exit_time]['ltp']
            pnl = round(exit_price - ltp, 2)
            trades.append((symbol, current_time, "BUY", ltp, exit_price, pnl))

        elif ltp < low and ask_qty > bid_qty:
            exit_time = timestamps[i + HOLDING_PERIOD * 60]
            exit_price = df.loc[exit_time]['ltp']
            pnl = round(ltp - exit_price, 2)
            trades.append((symbol, current_time, "SELL", ltp, exit_price, pnl))

    return trades

# Run on all CSVs
for file in os.listdir(FOLDER):
    if not file.endswith(".csv"):
        continue
    symbol = file.replace(".csv", "")
    path = os.path.join(FOLDER, file)

    try:
        df = load_tick_data(path)
        trades = simulate_ltp_breakouts(df, symbol)
        total_pnl = round(sum(t[5] for t in trades), 2)
        results.append((symbol, len(trades), total_pnl))
    except Exception as e:
        results.append((symbol, 0, 0.0))

# Display results
results.sort(key=lambda x: x[2], reverse=True)
print("\n📈 NIFTY50 LTP Breakout Strategy Backtest:\n")
for sym, n, pnl in results:
    print(f"{sym:<12} | Trades: {n:<3} | PnL: ₹{pnl:.2f}")

print("\n🔚 Summary")
print(f"📊 Total Trades: {sum(x[1] for x in results)}")
print(f"💰 Net P&L: ₹{round(sum(x[2] for x in results), 2)}")



📈 NIFTY50 LTP Breakout Strategy Backtest:

BHARTIARTL   | Trades: 0   | PnL: ₹0.00
TATASTEEL    | Trades: 0   | PnL: ₹0.00
TITAN        | Trades: 0   | PnL: ₹0.00
BRITANNIA    | Trades: 0   | PnL: ₹0.00
HDFCLIFE     | Trades: 0   | PnL: ₹0.00
COALINDIA    | Trades: 0   | PnL: ₹0.00
JSWSTEEL     | Trades: 0   | PnL: ₹0.00
NESTLEIND    | Trades: 0   | PnL: ₹0.00
ICICIBANK    | Trades: 0   | PnL: ₹0.00
MARUTI       | Trades: 0   | PnL: ₹0.00
ULTRACEMCO   | Trades: 0   | PnL: ₹0.00
WIPRO        | Trades: 0   | PnL: ₹0.00
NTPC         | Trades: 0   | PnL: ₹0.00
ASIANPAINT   | Trades: 0   | PnL: ₹0.00
ONGC         | Trades: 0   | PnL: ₹0.00
IOC          | Trades: 0   | PnL: ₹0.00
DRREDDY      | Trades: 0   | PnL: ₹0.00
TECHM        | Trades: 0   | PnL: ₹0.00
TCS          | Trades: 0   | PnL: ₹0.00
SUNPHARMA    | Trades: 0   | PnL: ₹0.00
HCLTECH      | Trades: 0   | PnL: ₹0.00
KOTAKBANK    | Trades: 0   | PnL: ₹0.00
AXISBANK     | Trades: 0   | PnL: ₹0.00
EICHERMOT    | Trades: 0   | PnL: ₹0